In [3]:
import sys
import argparse
import numpy as np
from PIL import Image
from io import BytesIO
import requests

from keras.preprocessing import image

# from keras import Input
# from keras.utils import np_utils
# from keras.utils.data_utils import get_file
from keras.preprocessing import image 
# from keras.applications import InceptionV3
from keras.applications.inception_v3 import preprocess_input
# from keras.models import Model,load_model
# from keras.layers import Dense, Dropout, Concatenate
# from keras.optimizers import SGD 
# import requests
# import numpy as np
# import pandas as pd
import json
import h5py
from PIL import Image
from keras.models import load_model
from collections import Counter
# load json and create model
model = load_model('my_model_0.h5')


In [16]:
target_size = (320, 320)

def decode_predictions(_predictions, top):
    tag_to_num = {'#WolfMillionaire': 0, '#MadWhips': 1, '#ad': 2, '#liketkit': 3, '#travel': 4, '#nature': 5, '#sponsored': 6, '#beautifuldestinations': 7, '#architecture': 8, '#love': 9, '#ootd': 10, '#fashion': 11, '#instagood': 12, '#onassignment': 13, '#coachella': 14, '#photooftheday': 15, '#tbt': 16, '#vegan': 17, '#fitness': 18, '#photography': 19, '#design': 20, '#artofvisuals': 21, '#Repost': 22, '#nyc': 23, '#earthday': 24, '#art': 25, '#conservation': 26, '#natgeo': 27, '#yoga': 28, '#beauty': 29, '#photojournalism': 30, '#paris': 31, '#coachella2017': 32, '#europe': 33, '#bestwoof': 34, '#beautiful': 35, '#moodygrams': 36, '#style': 37, '#ig_europe': 38, '#wonderful_places': 39, '#spring': 40, '#interiordesign': 41, '#sonyalpha': 42, '#travelblogger': 43, '#picoftheday': 44, '#istanbul': 45, '#living_europe': 46, '#breakfast': 47, '#worldplaces': 48, '#Coachella': 49, '#berlin': 50, '#instalike': 51, '#lifestyle': 52, '#Italy': 53, '#exploretocreate': 54, '#smartfit': 55, '#agameoftones': 56, '#LA': 57, '#africa': 58, '#nikonambassador': 59, '#newyork': 60, '#vacationwolf': 61, '#MusikaFrere': 62, '#canada': 63, '#ocean': 64, '#ig_europa': 65, '#foodphotography': 66, '#streetdreamsmag': 67, '#diewocheaufinstagram': 68, '#food': 69, '#alldaytravel': 70, '#climatechange': 71, '#Audi': 72, '#followme': 73, '#vsco': 74, '#beautifulcuisines': 75, '#bali': 76, '#ihavethisthingwithfloors': 77, '#bladelife': 78, '#knives': 79, '#watchthisinstagood': 80, '#adventure': 81, '#wildernessculture': 82, '#turkey': 83, '#instalove': 84, '#Architecture': 85, '#earthpix': 86, '#wdywt': 87, '#instago': 88, '#Nissan': 89, '#makeup': 90, '#luxury': 91, '#discoverearth': 92, '#EarthDay': 93, '#NYC': 94, '#indonesia': 95, '#plantbased': 96, '#follow': 97, '#germany': 98, '#Porsche': 99, '#mayabeaute': 100, '#whatitalyis': 101, '#wanderlust': 102, '#itsistanbul': 103, '#workout': 104, '#instatravel': 105, '#wildlife': 106, '#AMG': 107, '#travelblog': 108, '#huffpostgram': 109, '#chocolate': 110, '#visualsoflife': 111, '#NissanGTR': 112, '#urbanandstreet': 113, '#topeuropephoto': 114, '#sunset': 115, '#nikonlove': 116, '#vintage': 117, '#home': 118, '#cave': 119, '#healthy': 120, '#wu_europe': 121, '#TLPicks': 122, '#instagram': 123, '#Paris': 124, '#foodie': 125, '#visualambassadors': 126, '#vscocam': 127, '#homedesign': 128, '#landscape': 129, '#nikonnofilter': 130, '#TBT': 131, '#tattoo': 132, '#inspiration': 133, '#auto': 134, '#selfie': 135, '#lifestyleblogger': 136, '#thatsdarling': 137, '#passionpassport': 138, '#openmyworld': 139, '#instadaily': 140, '#travelling': 141, '#RCMemories': 142, '#marrakech': 143, '#morocco': 144, '#healthyfood': 145, '#astonmartin': 146, '#motivation': 147, '#delicious': 148, '#hbouthere': 149, '#traveldeeper': 150, '#hotel': 151, '#menswear': 152, '#goals': 153, '#me': 154, '#california': 155, '#aov': 156, '#greece': 157, '#drivetastefully': 158, '#GTR': 159, '#maybellineitgirls': 160, '#earthfocus': 161, '#amazing': 162, '#italy_vacations': 163, '#tattoos': 164, '#hello_france': 165, '#pulpriothair': 166, '#Pagani': 167, '#createcommune': 168, '#igers': 169, '#weekend': 170, '#urbanromantix': 171, '#phototag_it': 172, '#bestvacations': 173, '#easter': 174, '#revolvefestival': 175, '#china': 176, '#italy': 177, '#happy': 178, '#model': 179, '#fantastic_earth': 180, '#classic': 181, '#icon': 182, '#trip': 183, '#PatagoniaStory': 184, '#red': 185, '#2017': 186, '#pnwonderland': 187, '#brazilianbondbuilder': 188, '#cbviews': 189, '#way2ill': 190, '#menwithclass': 191, '#festivalfierce': 192, '#veganfood': 193, '#vanlifers': 194, '#6sense': 195, '#frasi': 196, '#foodandwine': 197, '#feedfeed': 198, '#australia': 199, '#archilovers': 200, '#archixxi': 201, '#bevisuallyinspired': 202, '#audi_obsession': 203, '#audidriven': 204, '#quattro': 205, '#crossfit': 206, '#BlakeScott': 207, '#GentsLounge': 208, '#explorecanada': 209, '#instacar': 210, '#carporn': 211, '#autoporn': 212, '#automotive': 213, '#drive': 214, '#classiccar': 215, '#collect': 216, '#classiccarchasers': 217, '#carphotography': 218, '#forsale': 219, '#crazyjewishmom': 220, '#d_signers': 221, '#California': 222, '#MercedesBenz': 223, '#McLaren': 224, '#DevanOnDeck': 225, '#DEVOURPOWER': 226, '#KarlitosWay': 227, '#FcukNormal': 228, '#maybellinepartner': 229, '#eternal_noir': 230, '#eternalnoir': 231, '#europe_vacations': 232, '#europe_gallery': 233, '#igerstravel': 234, '#bestdestinations': 235, '#discovereurope': 236, '#europe_focus_on': 237, '#super_europe': 238, '#besteuropephotos': 239, '#unlimitedeurope': 240, '#ok_europe': 241, '#loves_europe': 242, '#europaviagens': 243, '#destinationeurope': 244, '#europedestinations': 245, '#eurotrip': 246, '#europetrip': 247, '#eurotrip2017': 248, '#traveleurope': 249, '#eurotravel': 250, '#everlookphotography': 251, '#ExoticsBrazil': 252, '#fixthisbuildthat': 253, '#frangocombatatadoce': 254, '#frasesdem3rda': 255, '#gatherandfeast': 256, '#traveltheworld': 257, '#gurukafa': 258, '#jestemzolza': 259, '#zolza': 260, '#bezlitosne': 261, '#zyciowe': 262, '#typowe': 263, '#dieta': 264, '#myinstax': 265, '#noairneeded': 266, '#kostawilliams': 267, '#leagueofupdates': 268, '#leagueoflegends': 269, '#mtl': 270, '#montreal': 271, '#livemontreal': 272, '#minilicious': 273, '#fashionkids': 274, '#kidsfashion': 275, '#YourShotPhotographer': 276, '#fujifilm': 277, '#nicanorgarcia': 278, '#pilotpatrick': 279, '#intimasdaray': 280, '#restlessarch': 281, '#reza': 282, '#rezaphoto': 283, '#rezadeghati': 284, '#rezaphotography': 285, '#rezaphotojournalist': 286, '#MrAMG': 287, '#RokenR': 288, '#secdus': 289, '#sefayamak': 290, '#instagramturkey': 291, '#SolelyNikes': 292, '#StrictlyWeddings': 293, '#SwissWatches': 294, '#teamfk': 295, '#theshadowconspiracy': 296, '#bmx': 297, '#teamshape': 298, '#weddingz': 299, '#family': 300, '#streetsofturin': 301, '#placesofturin': 302, '#urbanlife': 303, '#neverstopexploring': 304, '#sunrise': 305, '#Ad': 306, '#alenaakhmadullina': 307, '#elephant': 308, '#photoark': 309, '#sp': 310, '#Gentleman': 311, '#TravelwithTJD': 312, '#cat': 313, '#barcelona': 314, '#exploration': 315, '#rsthecon': 316, '#tourcanada': 317, '#explorebc': 318, '#canadasworld': 319, '#ilovecanada': 320, '#enjoycanada': 321, '#christendominique': 322, '#dailyfoodfeed': 323, '#awesome': 324, '#health': 325, '#like': 326, '#Stockholm': 327, '#HB4L': 328, '#ILoveNSW': 329, '#houseofleaders': 330, '#indian_wedding_inspiration': 331, '#59in52': 332, '#NPS100': 333, '#fujifilmx_us': 334, '#myfujifilm': 335, '#XT1': 336, '#teamcozy': 337, '#ilovetravel': 338, '#getaway': 339, '#lojain': 340, '#lojain_omran': 341, '#lojain_style': 342, '#lojain_lifestyle': 343, '#lojain_world': 344, '#lovelypepatravels': 345, '#instafood': 346, '#expedition': 347, '#kidsofinstagram': 348, '#ig_masterpiece': 349, '#eroticart': 350, '#erotic': 351, '#petitesluxures': 352, '#popeyethefoodie': 353, '#webistan': 354, '#santantactical': 355, '#sheikestyle': 356, '#explorer': 357, '#shoutmysneakz': 358, '#SolelyJumpman': 359, '#igersaustria': 360, '#bride': 361, '#suchar': 362, '#codzienny': 363, '#sucharcodzienny': 364, '#TATTOODO': 365, '#tgrlivethedream': 366, '#CharlotteGroeneveld': 367, '#Thefashionguitar': 368, '#thegoodquote': 369, '#universityofrock': 370, '#uolesporte': 371, '#weddingzin': 372, '#academia': 373, '#shoes': 374, '#girls': 375, '#lookslikefilm': 376, '#foodstyling': 377, '#flowers': 378, '#traveling': 379, '#travelstoke': 380, '#miami': 381, '#fitspo': 382, '#beach': 383, '#Arc_Only': 384, '#alaska': 385, '#hallazgosemanal': 386, '#spain': 387, '#exploremore': 388, '#thailand': 389, '#underwater': 390, '#cute': 391, '#boosted': 392, '#communityfirst': 393, '#trippinwithtarte': 394, '#performance': 395, '#deemabayyaa': 396, '#menwithstreetstyle': 397, '#edmHumor': 398, '#awesomeglobe': 399, '#AutogespotBrazil': 400, '#Lamborghini': 401, '#like4like': 402, '#Hairbestie': 403, '#27A': 404, '#Amsterdam': 405, '#kobieta': 406, '#lovewatts': 407, '#hskicks': 408, '#hypefeet': 409, '#la': 410, '#tourist': 411, '#sonyalphatr': 412, '#1dapperst': 413, '#qualmedie': 414, '#mattiasklum': 415, '#mikeleceppi': 416, '#movified': 417, '#createexplore': 418, '#ig_mood': 419, '#lifeofadventure': 420, '#eroticdrawing': 421, '#StayWow': 422, '#travelrack': 423, '#wonderful_location': 424, '#uol': 425, '#YSeCallan': 426, '#weddingphotography': 427, '#musculacao': 428, '#treino': 429, '#crossfitgirls': 430, '#projetobabababy': 431, '#crossfitbrasil': 432, '#mashpics': 433, '#interior': 434, '#f52grams': 435, '#interiors': 436, '#decorating': 437, '#landscapedesign': 438, '#stunning': 439, '#usa': 440, '#kenya': 441, '#bikini': 442, '#blue': 443, '#Japan': 444, '#EmiratesAirlines': 445, '#awesomedreamplaces': 446, '#effectivehandstands': 447, '#freshairandfreedom': 448, '#explore': 449, '#healthylifestyle': 450, '#bloggermexicana': 451, '#streetlife': 452, '#gratitude': 453, '#weroamgermany': 454, '#chile': 455, '#MercedesAMG': 456, '#eatandshout': 457, '#iphoneonly': 458, '#BeYouBeTrue': 459, '#mytravelgram': 460, '#Hairbestiesforlife': 461, '#Evolve': 462, '#heidimakeupartist': 463, '#pictureoftheday': 464, '#chootravels': 465, '#seamsforadesire': 466, '#travelingram': 467, '#ColoradoLive': 468, '#boostheaven': 469, '#adidasvibes': 470, '#sneakersaddict': 471, '#snkrhds': 472, '#crepecity': 473, '#minimalmovement': 474, '#thesolesupplier': 475, '#praisemag': 476, '#passportready': 477, '#instapassport': 478, '#igtravel': 479, '#travelpics': 480, '#travelphoto': 481, '#lizaonair': 482, '#m_eye_nd': 483, '#SeeMyCity': 484, '#protectbiodiversity': 485, '#frases': 486, '#inspiraci': 487, '#FOLLOWMETO': 488, '#heatercentral': 489, '#pixeltheplanet': 490, '#soysaludable': 491, '#canonusa': 492, '#TheClassyPeople': 493, '#VacationWolf': 494, '#IAmAnAlly': 495, '#vidadecrossfiteiro': 496, '#beldadesdocrossfit': 497, '#whole30brasil': 498, '#hair': 499, '#mobilemag': 500, '#achievetheimpossible': 501, '#wonderfulindonesia': 502, '#jalanjalan': 503, '#nationalgeographic': 504, '#mensfashion': 505, '#relax': 506, '#summer': 507, '#happiness': 508, '#healthyliving': 509, '#sea': 510, '#Coachella2017': 511, '#hawaii': 512, '#aleutiandreams': 513, '#4rings': 514, '#drivenbyvorsprung': 515, '#portrait': 516, '#decoration': 517, '#smile': 518, '#quotes': 519, '#ChEatFamousFriday': 520, '#utah': 521, '#lonelyplanet': 522, '#Ferrari': 523, '#bengali': 524, '#friday': 525, '#dessert': 526, '#wildlifephotography': 527, '#wonderfulplaces': 528, '#goodmorning': 529, '#mydentity': 530, '#Mydentity': 531, '#hairgod_zito': 532, '#necklace': 533, '#ifbbpro': 534, '#IFLW': 535, '#tiles': 536, '#highjewellery': 537, '#chicago': 538, '#practiceyogachangeyourworld': 539, '#onebreathatatime': 540, '#steelarms': 541, '#knifeaxe': 542, '#blades': 543, '#bladeporn': 544, '#military': 545, '#karambit': 546, '#knifenut': 547, '#Scalpel': 548, '#Machete': 549, '#Axe': 550, '#katana': 551, '#blade': 552, '#sword': 553, '#knifestagram': 554, '#knifepics': 555, '#knifelife': 556, '#knifegasm': 557, '#knifecollector': 558, '#Knife': 559, '#knifecollection': 560, '#customknives': 561, '#knifenuts': 562, '#knivesofig': 563, '#knifecommunity': 564, '#knivesofinstagram': 565, '#survivalknife': 566, '#visiting': 567, '#illgrammers': 568, '#Queen_of_the_Arabian_kitchen': 569, '#manalalalem': 570, '#chef_celebrity': 571, '#recipes': 572, '#dish': 573, '#kerinaootd': 574, '#motivaci': 575, '#cookbook2': 576, '#fatalframes': 577, '#petfriendlydining': 578, '#heresmyfood': 579, '#stretch': 580, '#tracknation': 581, '#secdusshop': 582, '#weekendinthedesert': 583, '#sisterlystyle': 584, '#Foodblogger': 585, '#swpartners': 586, '#Itswhitenoise': 587, '#MeVolviADesviarDelTema': 588, '#MrWycked': 589, '#desafio40diasnofoco': 590, '#secavaca': 591, '#myrbk': 592, '#cfbroficial': 593, '#tanquinho': 594, '#saopaulo': 595, '#vscogood': 596, '#ourfoodstories': 597, '#throwback': 598, '#abanddoned': 599, '#france': 600, '#amivitale': 601, '#anadelia': 602, '#strong': 603, '#Tokyo': 604, '#commercialfishing': 605, '#losangeles': 606, '#artofdrawingg': 607, '#breathe': 608, '#black': 609, '#green': 610, '#keepitwild': 611, '#currentlywearing': 612, '#getoutside': 613, '#car': 614, '#wearesummer17': 615, '#mexicanbloggers': 616, '#staystrong': 617, '#muscle': 618, '#fun': 619, '#myauvergne': 620, '#FranceFR': 621, '#lighting': 622, '#InnerForce': 623, '#streetstyle': 624, '#quote': 625, '#EATINGNYC': 626, '#trendofpeople': 627, '#palmspring': 628, '#discoverglobe': 629, '#natgeotravel': 630, '#bbctravel': 631, '#faheeym': 632, '#fittriocommunity': 633, '#welovegoldens': 634, '#main_vision': 635, '#1': 636, '#followmeto': 637, '#hotdudesreading': 638, '#bodybuilding': 639, '#mercedesbenz': 640, '#exklusive_shot': 641, '#runaflirts': 642, '#jaefit': 643, '#prawda': 644, '#writetotravel': 645, '#instavacation': 646, '#postcardsfromtheworld': 647, '#travelstroke': 648, '#JustAnotherDayInWA': 649, '#bmw': 650, '#gtr': 651, '#easyrecipes': 652, '#easyfood': 653, '#MarioTestino': 654, '#hijadecristo': 655, '#eatbetternotlesstravels': 656, '#igmasters': 657, '#estheticlabel': 658, '#veganfoodshare': 659, '#bookstagram': 660, '#eatmoreplants': 661, '#healthylife': 662, '#runnerspace': 663, '#guns': 664, '#pgdaily': 665, '#kirstentibballs': 666, '#callebaut': 667, '#StephenWilkes': 668, '#redcine': 669, '#ldfoundation': 670, '#Carinstagram': 671, '#kings_villages': 672, '#LGBTQI': 673, '#whereloveisillegal': 674, '#reebokcrossfit': 675, '#desafiofitem2017': 676, '#happyeaster': 677, '#live': 678, '#exittostreet': 679, '#glutenfree': 680, '#RECO_ig': 681, '#tb': 682, '#interiordecor': 683, '#livelife': 684, '#enjoy': 685, '#chamlive': 686, '#stayandwander': 687, '#travelawesome': 688, '#vscogram': 689, '#naturelovers': 690, '#iphone7': 691, '#springstyle': 692, '#magicalkenya': 693, '#nikonusa': 694, '#nikond5s': 695, '#oregon': 696, '#milano': 697, '#andreleonardo': 698, '#linkinbio': 699, '#japan': 700, '#USA': 701, '#SaltonSea': 702, '#carsbyaudisport': 703, '#ourplanetdaily': 704, '#thesummerexpedition': 705, '#PopUpPlayground': 706, '#Berlin': 707, '#travelgram': 708, '#ireland': 709, '#whpnaturalbeauty': 710, '#coffee': 711, '#vanlife': 712, '#g': 713, '#Easter': 714, '#abs': 715, '#cats': 716, '#fbf': 717, '#chicagofoodauthority': 718, '#toofacedxquay': 719, '#Leica': 720, '#lovefood': 721, '#thekitchn': 722, '#huffposttaste': 723, '#architect': 724, '#decor': 725, '#women': 726, '#moreocean': 727, '#fitnessfood': 728, '#birds': 729, '#EatFamous': 730, '#potd': 731, '#BMW': 732, '#fozazastyle': 733, '#garanceinjapan': 734, '#igersfrance': 735, '#bestdiscovery': 736, '#vienna': 737, '#earthofficial': 738, '#loves_paris': 739, '#parismaville': 740, '#hotelsandresorts': 741, '#Altamira': 742, '#ireneismaldives': 743, '#canon_photos': 744, '#bequadzilla': 745, '#mensphysique': 746, '#karinanigay': 747, '#finejewellery': 748, '#KERRentlyWearing': 749, '#sneakerfreakerfam': 750, '#sneakerlove': 751, '#bor': 752, '#TMM': 753, '#MommyAndMini2017': 754, '#fashionblogger': 755, '#super_france': 756, '#maserati': 757, '#badass': 758, '#menwithfootwear': 759, '#Mercurio': 760, '#Venus': 761, '#ltkunder50': 762, '#mkexplore': 763, '#splendid_shotz': 764, '#bestmeow': 765, '#veganrecipes': 766, '#govegan': 767, '#usatf': 768, '#IamATraveler': 769, '#savourschool': 770, '#EntrepreneurLife': 771, '#fashionworld': 772, '#fashiongirls': 773, '#nglive': 774, '#BCI': 775, '#bigcatsintiative': 776, '#weddingideas': 777, '#instabride': 778, '#weddinginspo': 779, '#guardiantravelsnaps': 780, '#projetoverao2017': 781, '#designporn': 782, '#czechoutmoravia': 783, '#wanderaround': 784, '#lovemylife': 785, '#indonesiaku': 786, '#resort': 787, '#nikonindonesia': 788, '#iamindonesia': 789, '#nikonasia': 790, '#mynikonlife': 791, '#nikonaustralia': 792, '#ITSistanbul': 793, '#vscodaily': 794, '#fw1718': 795, '#saveelephants': 796, '#retetielephants': 797, '#stoppoaching': 798, '#northernkenya': 799, '#whyilovekenya': 800, '#everydayafrica': 801, '#whale': 802, '#iceland': 803, '#girl': 804, '#danielwellington': 805, '#details': 806, '#pancakes': 807, '#winter': 808, '#peace': 809, '#aerial': 810, '#ltkunder100': 811, '#passion': 812, '#lifestyleblog': 813, '#thehappynow': 814, '#palmsprings': 815, '#zenfone3zoom': 816, '#captureomomento': 817, '#dubai': 818, '#tastingtable': 819, '#latergram': 820, '#movementtherapy': 821, '#designer': 822, '#womensbest': 823, '#exploreeverything': 824, '#liveinlevis': 825, '#cool': 826, '#light': 827, '#PanjiRasulullah': 828, '#foodporn': 829, '#movies': 830, '#parisjetaime': 831, '#topparisphoto': 832, '#onehairdressing': 833, '#fromwhereistand': 834, '#WhereLoveIsIllegal': 835, '#WalkingtheLAwash': 836, '#gotgvol2': 837, '#polishgirl': 838, '#nigay_ny': 839, '#hautejoaillerie': 840, '#thefateofthefurious': 841, '#sonyalphasclub': 842, '#theartofslowliving': 843, '#ford': 844, '#Huayra': 845, '#Luxury': 846, '#insadelicious': 847, '#manizha': 848, '#childofgod': 849, '#science': 850, '#inflowtravel': 851, '#streetmagazine': 852, '#hot_shotz': 853, '#moodytoning': 854, '#lifeandthyme': 855, '#Toronto': 856, '#balance': 857, '#flexible': 858, '#rescuefamily': 859, '#fashionpic': 860, '#fashiontips': 861, '#fashionphoto': 862, '#fashionhair': 863, '#fashionismo': 864, '#fashionbook': 865, '#fashionnews': 866, '#fashionbrand': 867, '#fashionshop': 868, '#fursforlife': 869, '#startwith1thing': 870, '#instawedding': 871, '#yourshotphotographer': 872, '#LoveFL': 873, '#thisisgoodcompany': 874, '#veganrecipe': 875, '#natgeohub': 876, '#TeamWycked': 877, '#dress': 878, '#happygirl': 879, '#party': 880, '#friends': 881, '#tv_buildings': 882, '#eattheworld': 883, '#travelwithme': 884, '#visitCZ': 885, '#ontheroad': 886, '#canon': 887, '#vietnam': 888, '#work': 889, '#sydney': 890, '#natureisspeaking': 891, '#india': 892, '#woman': 893, '#puertorico': 894, '#transformationtuesday': 895, '#alphacollective': 896, '#pretty': 897, '#outfit': 898, '#London': 899, '#america': 900, '#R8': 901, '#london': 902, '#BarcelonaCitizen': 903, '#igersbcn': 904, '#catalonia': 905, '#parkguell': 906, '#botswana': 907, '#nordstrom': 908, '#mountaingirls': 909, '#gopro': 910, '#carlifestyle': 911, '#blogdebelleza': 912, '#modernclassic': 913, '#bloglovinfood': 914, '#sonya7rii': 915, '#grateful': 916, '#E63S': 917, '#frankfurt': 918, '#chakra': 919, '#inabudhabi': 920, '#documentary': 921, '#newyorkcity': 922, '#cookbook': 923, '#jordan': 924, '#nakedplanet': 925, '#faafif': 926, '#MasirahPanjiRasulullah': 927, '#IslamRahmatanLilAlamin': 928, '#futuresolutionlx': 929, '#capetown': 930, '#LosAngeles': 931, '#topfrancephoto': 932, '#awesomepix': 933, '#natgeolandscape': 934, '#igersparis': 935, '#fashionstyling': 936, '#tat': 937, '#tatoo': 938, '#besttattoos': 939, '#ideastattoo': 940, '#blogger': 941, '#wbffpro': 942, '#highjewelry': 943, '#blackpepperenigma': 944, '#boostgod': 945, '#boostVIBES': 946, '#boostGANG': 947, '#modernnotoriety': 948, '#sonyimages': 949, '#PorscheGT3RS': 950, '#PaganiHuayra': 951, '#Zonda': 952, '#PaganiZonda': 953, '#makaniterror': 954, '#Jewelry': 955, '#vegetarian': 956, '#fashiondiaries': 957, '#Urano': 958, '#poetry': 959, '#modaparameninas': 960, '#instapic': 961, '#pracegover': 962, '#newdarlingsHOME': 963, '#weekly_feature': 964, '#fartoodope': 965, '#usaprimeshot': 966, '#master_shots': 967, '#royalsnappingartists': 968, '#transfer_visions': 969, '#philippemodel': 970, '#sunday': 971, '#whatveganseat': 972, '#ar15': 973, '#ourplanet': 974, '#Allshotsturkey': 975, '#bir_dakika': 976, '#gununkaresi': 977, '#hayatakarken': 978, '#turkinstagram': 979, '#zamanidurdur': 980, '#instasyon': 981, '#istanbuldayasam': 982, '#beniminsanlarim': 983, '#bugununkaresi': 984, '#ig_turkey': 985, '#turkishfollowers': 986, '#objektifimden': 987, '#my_dream_turkey': 988, '#gf_turkey': 989, '#igersturkey': 990, '#igersistanbul': 991, '#fashionnails': 992, '#fashionstudent': 993, '#breaky': 994, '#vscofashionfood': 995, '#edinburgh': 996, '#theworldguru': 997, '#MyFloridaisAdventure': 998, '#vegancommunity': 999}
    counts = Counter(tag_to_num)
    counts = sorted(counts, key=counts.__getitem__,reverse=True)[:1000]
#     print(_predictions)
    preds =  _predictions[0].tolist()[:1000]
    data = zip(counts, preds)
    return sorted(data,key = lambda t: t[1])[:5]

def predict_object(model, img, target_size, top_n=5):
    if img.size != target_size:
        img = img.resize(target_size)
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    _predictions = model.predict(x)
    return decode_predictions(_predictions, top=top_n)

path = '../../ml/wedding/new_pics/DSC_0001.JPG'
img = Image.open(path)
predictions = predict_object(model, img, target_size)
print(predictions)


[[ -9.57167298e-02   2.84440488e-01   2.55112290e-01  -7.34080732e-01
    5.91070205e-02   9.10920948e-02  -6.21551096e-01   4.35882270e-01
   -8.21085751e-01   4.94046330e-01   2.53682286e-01  -3.66174787e-01
   -2.02142000e-01  -1.63186520e-01  -6.05792880e-01  -7.68552780e-01
    6.78251982e-02   7.25119174e-01  -5.72551414e-02  -5.72219901e-02
    2.14733705e-01   5.02829850e-01  -2.28977203e-01   5.30669689e-01
   -2.25956082e-01   4.08961713e-01  -2.85833269e-01   1.06848514e+00
   -4.78474617e-01  -4.33658004e-01   5.80737777e-02   8.84711206e-01
    3.59277055e-02  -1.10947244e-01   4.52518672e-01   2.30788648e-01
   -3.44352961e-01   7.39720643e-01  -1.04443364e-01   9.08918381e-02
    3.49242508e-01   4.65162009e-01  -8.64482205e-03   2.25721136e-01
    1.25453295e-02  -1.02933139e-01  -4.60783750e-01   3.67123127e-01
    1.08177923e-01   1.35404961e-02  -1.22487664e-01  -4.35839184e-02
   -5.73290050e-01   6.61597073e-01   4.23708797e-01  -9.00940716e-01
   -1.43900320e-01  

In [8]:
target_size = (320, 320)

def decode_predictions(_predictions, top):
    tag_to_num = {'#WolfMillionaire': 0, '#MadWhips': 1, '#ad': 2, '#liketkit': 3, '#travel': 4, '#nature': 5, '#sponsored': 6, '#beautifuldestinations': 7, '#architecture': 8, '#love': 9, '#ootd': 10, '#fashion': 11, '#instagood': 12, '#onassignment': 13, '#coachella': 14, '#photooftheday': 15, '#tbt': 16, '#vegan': 17, '#fitness': 18, '#photography': 19, '#design': 20, '#artofvisuals': 21, '#Repost': 22, '#nyc': 23, '#earthday': 24, '#art': 25, '#conservation': 26, '#natgeo': 27, '#yoga': 28, '#beauty': 29, '#photojournalism': 30, '#paris': 31, '#coachella2017': 32, '#europe': 33, '#bestwoof': 34, '#beautiful': 35, '#moodygrams': 36, '#style': 37, '#ig_europe': 38, '#wonderful_places': 39, '#spring': 40, '#interiordesign': 41, '#sonyalpha': 42, '#travelblogger': 43, '#picoftheday': 44, '#istanbul': 45, '#living_europe': 46, '#breakfast': 47, '#worldplaces': 48, '#Coachella': 49, '#berlin': 50, '#instalike': 51, '#lifestyle': 52, '#Italy': 53, '#exploretocreate': 54, '#smartfit': 55, '#agameoftones': 56, '#LA': 57, '#africa': 58, '#nikonambassador': 59, '#newyork': 60, '#vacationwolf': 61, '#MusikaFrere': 62, '#canada': 63, '#ocean': 64, '#ig_europa': 65, '#foodphotography': 66, '#streetdreamsmag': 67, '#diewocheaufinstagram': 68, '#food': 69, '#alldaytravel': 70, '#climatechange': 71, '#Audi': 72, '#followme': 73, '#vsco': 74, '#beautifulcuisines': 75, '#bali': 76, '#ihavethisthingwithfloors': 77, '#bladelife': 78, '#knives': 79, '#watchthisinstagood': 80, '#adventure': 81, '#wildernessculture': 82, '#turkey': 83, '#instalove': 84, '#Architecture': 85, '#earthpix': 86, '#wdywt': 87, '#instago': 88, '#Nissan': 89, '#makeup': 90, '#luxury': 91, '#discoverearth': 92, '#EarthDay': 93, '#NYC': 94, '#indonesia': 95, '#plantbased': 96, '#follow': 97, '#germany': 98, '#Porsche': 99, '#mayabeaute': 100, '#whatitalyis': 101, '#wanderlust': 102, '#itsistanbul': 103, '#workout': 104, '#instatravel': 105, '#wildlife': 106, '#AMG': 107, '#travelblog': 108, '#huffpostgram': 109, '#chocolate': 110, '#visualsoflife': 111, '#NissanGTR': 112, '#urbanandstreet': 113, '#topeuropephoto': 114, '#sunset': 115, '#nikonlove': 116, '#vintage': 117, '#home': 118, '#cave': 119, '#healthy': 120, '#wu_europe': 121, '#TLPicks': 122, '#instagram': 123, '#Paris': 124, '#foodie': 125, '#visualambassadors': 126, '#vscocam': 127, '#homedesign': 128, '#landscape': 129, '#nikonnofilter': 130, '#TBT': 131, '#tattoo': 132, '#inspiration': 133, '#auto': 134, '#selfie': 135, '#lifestyleblogger': 136, '#thatsdarling': 137, '#passionpassport': 138, '#openmyworld': 139, '#instadaily': 140, '#travelling': 141, '#RCMemories': 142, '#marrakech': 143, '#morocco': 144, '#healthyfood': 145, '#astonmartin': 146, '#motivation': 147, '#delicious': 148, '#hbouthere': 149, '#traveldeeper': 150, '#hotel': 151, '#menswear': 152, '#goals': 153, '#me': 154, '#california': 155, '#aov': 156, '#greece': 157, '#drivetastefully': 158, '#GTR': 159, '#maybellineitgirls': 160, '#earthfocus': 161, '#amazing': 162, '#italy_vacations': 163, '#tattoos': 164, '#hello_france': 165, '#pulpriothair': 166, '#Pagani': 167, '#createcommune': 168, '#igers': 169, '#weekend': 170, '#urbanromantix': 171, '#phototag_it': 172, '#bestvacations': 173, '#easter': 174, '#revolvefestival': 175, '#china': 176, '#italy': 177, '#happy': 178, '#model': 179, '#fantastic_earth': 180, '#classic': 181, '#icon': 182, '#trip': 183, '#PatagoniaStory': 184, '#red': 185, '#2017': 186, '#pnwonderland': 187, '#brazilianbondbuilder': 188, '#cbviews': 189, '#way2ill': 190, '#menwithclass': 191, '#festivalfierce': 192, '#veganfood': 193, '#vanlifers': 194, '#6sense': 195, '#frasi': 196, '#foodandwine': 197, '#feedfeed': 198, '#australia': 199, '#archilovers': 200, '#archixxi': 201, '#bevisuallyinspired': 202, '#audi_obsession': 203, '#audidriven': 204, '#quattro': 205, '#crossfit': 206, '#BlakeScott': 207, '#GentsLounge': 208, '#explorecanada': 209, '#instacar': 210, '#carporn': 211, '#autoporn': 212, '#automotive': 213, '#drive': 214, '#classiccar': 215, '#collect': 216, '#classiccarchasers': 217, '#carphotography': 218, '#forsale': 219, '#crazyjewishmom': 220, '#d_signers': 221, '#California': 222, '#MercedesBenz': 223, '#McLaren': 224, '#DevanOnDeck': 225, '#DEVOURPOWER': 226, '#KarlitosWay': 227, '#FcukNormal': 228, '#maybellinepartner': 229, '#eternal_noir': 230, '#eternalnoir': 231, '#europe_vacations': 232, '#europe_gallery': 233, '#igerstravel': 234, '#bestdestinations': 235, '#discovereurope': 236, '#europe_focus_on': 237, '#super_europe': 238, '#besteuropephotos': 239, '#unlimitedeurope': 240, '#ok_europe': 241, '#loves_europe': 242, '#europaviagens': 243, '#destinationeurope': 244, '#europedestinations': 245, '#eurotrip': 246, '#europetrip': 247, '#eurotrip2017': 248, '#traveleurope': 249, '#eurotravel': 250, '#everlookphotography': 251, '#ExoticsBrazil': 252, '#fixthisbuildthat': 253, '#frangocombatatadoce': 254, '#frasesdem3rda': 255, '#gatherandfeast': 256, '#traveltheworld': 257, '#gurukafa': 258, '#jestemzolza': 259, '#zolza': 260, '#bezlitosne': 261, '#zyciowe': 262, '#typowe': 263, '#dieta': 264, '#myinstax': 265, '#noairneeded': 266, '#kostawilliams': 267, '#leagueofupdates': 268, '#leagueoflegends': 269, '#mtl': 270, '#montreal': 271, '#livemontreal': 272, '#minilicious': 273, '#fashionkids': 274, '#kidsfashion': 275, '#YourShotPhotographer': 276, '#fujifilm': 277, '#nicanorgarcia': 278, '#pilotpatrick': 279, '#intimasdaray': 280, '#restlessarch': 281, '#reza': 282, '#rezaphoto': 283, '#rezadeghati': 284, '#rezaphotography': 285, '#rezaphotojournalist': 286, '#MrAMG': 287, '#RokenR': 288, '#secdus': 289, '#sefayamak': 290, '#instagramturkey': 291, '#SolelyNikes': 292, '#StrictlyWeddings': 293, '#SwissWatches': 294, '#teamfk': 295, '#theshadowconspiracy': 296, '#bmx': 297, '#teamshape': 298, '#weddingz': 299, '#family': 300, '#streetsofturin': 301, '#placesofturin': 302, '#urbanlife': 303, '#neverstopexploring': 304, '#sunrise': 305, '#Ad': 306, '#alenaakhmadullina': 307, '#elephant': 308, '#photoark': 309, '#sp': 310, '#Gentleman': 311, '#TravelwithTJD': 312, '#cat': 313, '#barcelona': 314, '#exploration': 315, '#rsthecon': 316, '#tourcanada': 317, '#explorebc': 318, '#canadasworld': 319, '#ilovecanada': 320, '#enjoycanada': 321, '#christendominique': 322, '#dailyfoodfeed': 323, '#awesome': 324, '#health': 325, '#like': 326, '#Stockholm': 327, '#HB4L': 328, '#ILoveNSW': 329, '#houseofleaders': 330, '#indian_wedding_inspiration': 331, '#59in52': 332, '#NPS100': 333, '#fujifilmx_us': 334, '#myfujifilm': 335, '#XT1': 336, '#teamcozy': 337, '#ilovetravel': 338, '#getaway': 339, '#lojain': 340, '#lojain_omran': 341, '#lojain_style': 342, '#lojain_lifestyle': 343, '#lojain_world': 344, '#lovelypepatravels': 345, '#instafood': 346, '#expedition': 347, '#kidsofinstagram': 348, '#ig_masterpiece': 349, '#eroticart': 350, '#erotic': 351, '#petitesluxures': 352, '#popeyethefoodie': 353, '#webistan': 354, '#santantactical': 355, '#sheikestyle': 356, '#explorer': 357, '#shoutmysneakz': 358, '#SolelyJumpman': 359, '#igersaustria': 360, '#bride': 361, '#suchar': 362, '#codzienny': 363, '#sucharcodzienny': 364, '#TATTOODO': 365, '#tgrlivethedream': 366, '#CharlotteGroeneveld': 367, '#Thefashionguitar': 368, '#thegoodquote': 369, '#universityofrock': 370, '#uolesporte': 371, '#weddingzin': 372, '#academia': 373, '#shoes': 374, '#girls': 375, '#lookslikefilm': 376, '#foodstyling': 377, '#flowers': 378, '#traveling': 379, '#travelstoke': 380, '#miami': 381, '#fitspo': 382, '#beach': 383, '#Arc_Only': 384, '#alaska': 385, '#hallazgosemanal': 386, '#spain': 387, '#exploremore': 388, '#thailand': 389, '#underwater': 390, '#cute': 391, '#boosted': 392, '#communityfirst': 393, '#trippinwithtarte': 394, '#performance': 395, '#deemabayyaa': 396, '#menwithstreetstyle': 397, '#edmHumor': 398, '#awesomeglobe': 399, '#AutogespotBrazil': 400, '#Lamborghini': 401, '#like4like': 402, '#Hairbestie': 403, '#27A': 404, '#Amsterdam': 405, '#kobieta': 406, '#lovewatts': 407, '#hskicks': 408, '#hypefeet': 409, '#la': 410, '#tourist': 411, '#sonyalphatr': 412, '#1dapperst': 413, '#qualmedie': 414, '#mattiasklum': 415, '#mikeleceppi': 416, '#movified': 417, '#createexplore': 418, '#ig_mood': 419, '#lifeofadventure': 420, '#eroticdrawing': 421, '#StayWow': 422, '#travelrack': 423, '#wonderful_location': 424, '#uol': 425, '#YSeCallan': 426, '#weddingphotography': 427, '#musculacao': 428, '#treino': 429, '#crossfitgirls': 430, '#projetobabababy': 431, '#crossfitbrasil': 432, '#mashpics': 433, '#interior': 434, '#f52grams': 435, '#interiors': 436, '#decorating': 437, '#landscapedesign': 438, '#stunning': 439, '#usa': 440, '#kenya': 441, '#bikini': 442, '#blue': 443, '#Japan': 444, '#EmiratesAirlines': 445, '#awesomedreamplaces': 446, '#effectivehandstands': 447, '#freshairandfreedom': 448, '#explore': 449, '#healthylifestyle': 450, '#bloggermexicana': 451, '#streetlife': 452, '#gratitude': 453, '#weroamgermany': 454, '#chile': 455, '#MercedesAMG': 456, '#eatandshout': 457, '#iphoneonly': 458, '#BeYouBeTrue': 459, '#mytravelgram': 460, '#Hairbestiesforlife': 461, '#Evolve': 462, '#heidimakeupartist': 463, '#pictureoftheday': 464, '#chootravels': 465, '#seamsforadesire': 466, '#travelingram': 467, '#ColoradoLive': 468, '#boostheaven': 469, '#adidasvibes': 470, '#sneakersaddict': 471, '#snkrhds': 472, '#crepecity': 473, '#minimalmovement': 474, '#thesolesupplier': 475, '#praisemag': 476, '#passportready': 477, '#instapassport': 478, '#igtravel': 479, '#travelpics': 480, '#travelphoto': 481, '#lizaonair': 482, '#m_eye_nd': 483, '#SeeMyCity': 484, '#protectbiodiversity': 485, '#frases': 486, '#inspiraci': 487, '#FOLLOWMETO': 488, '#heatercentral': 489, '#pixeltheplanet': 490, '#soysaludable': 491, '#canonusa': 492, '#TheClassyPeople': 493, '#VacationWolf': 494, '#IAmAnAlly': 495, '#vidadecrossfiteiro': 496, '#beldadesdocrossfit': 497, '#whole30brasil': 498, '#hair': 499, '#mobilemag': 500, '#achievetheimpossible': 501, '#wonderfulindonesia': 502, '#jalanjalan': 503, '#nationalgeographic': 504, '#mensfashion': 505, '#relax': 506, '#summer': 507, '#happiness': 508, '#healthyliving': 509, '#sea': 510, '#Coachella2017': 511, '#hawaii': 512, '#aleutiandreams': 513, '#4rings': 514, '#drivenbyvorsprung': 515, '#portrait': 516, '#decoration': 517, '#smile': 518, '#quotes': 519, '#ChEatFamousFriday': 520, '#utah': 521, '#lonelyplanet': 522, '#Ferrari': 523, '#bengali': 524, '#friday': 525, '#dessert': 526, '#wildlifephotography': 527, '#wonderfulplaces': 528, '#goodmorning': 529, '#mydentity': 530, '#Mydentity': 531, '#hairgod_zito': 532, '#necklace': 533, '#ifbbpro': 534, '#IFLW': 535, '#tiles': 536, '#highjewellery': 537, '#chicago': 538, '#practiceyogachangeyourworld': 539, '#onebreathatatime': 540, '#steelarms': 541, '#knifeaxe': 542, '#blades': 543, '#bladeporn': 544, '#military': 545, '#karambit': 546, '#knifenut': 547, '#Scalpel': 548, '#Machete': 549, '#Axe': 550, '#katana': 551, '#blade': 552, '#sword': 553, '#knifestagram': 554, '#knifepics': 555, '#knifelife': 556, '#knifegasm': 557, '#knifecollector': 558, '#Knife': 559, '#knifecollection': 560, '#customknives': 561, '#knifenuts': 562, '#knivesofig': 563, '#knifecommunity': 564, '#knivesofinstagram': 565, '#survivalknife': 566, '#visiting': 567, '#illgrammers': 568, '#Queen_of_the_Arabian_kitchen': 569, '#manalalalem': 570, '#chef_celebrity': 571, '#recipes': 572, '#dish': 573, '#kerinaootd': 574, '#motivaci': 575, '#cookbook2': 576, '#fatalframes': 577, '#petfriendlydining': 578, '#heresmyfood': 579, '#stretch': 580, '#tracknation': 581, '#secdusshop': 582, '#weekendinthedesert': 583, '#sisterlystyle': 584, '#Foodblogger': 585, '#swpartners': 586, '#Itswhitenoise': 587, '#MeVolviADesviarDelTema': 588, '#MrWycked': 589, '#desafio40diasnofoco': 590, '#secavaca': 591, '#myrbk': 592, '#cfbroficial': 593, '#tanquinho': 594, '#saopaulo': 595, '#vscogood': 596, '#ourfoodstories': 597, '#throwback': 598, '#abanddoned': 599, '#france': 600, '#amivitale': 601, '#anadelia': 602, '#strong': 603, '#Tokyo': 604, '#commercialfishing': 605, '#losangeles': 606, '#artofdrawingg': 607, '#breathe': 608, '#black': 609, '#green': 610, '#keepitwild': 611, '#currentlywearing': 612, '#getoutside': 613, '#car': 614, '#wearesummer17': 615, '#mexicanbloggers': 616, '#staystrong': 617, '#muscle': 618, '#fun': 619, '#myauvergne': 620, '#FranceFR': 621, '#lighting': 622, '#InnerForce': 623, '#streetstyle': 624, '#quote': 625, '#EATINGNYC': 626, '#trendofpeople': 627, '#palmspring': 628, '#discoverglobe': 629, '#natgeotravel': 630, '#bbctravel': 631, '#faheeym': 632, '#fittriocommunity': 633, '#welovegoldens': 634, '#main_vision': 635, '#1': 636, '#followmeto': 637, '#hotdudesreading': 638, '#bodybuilding': 639, '#mercedesbenz': 640, '#exklusive_shot': 641, '#runaflirts': 642, '#jaefit': 643, '#prawda': 644, '#writetotravel': 645, '#instavacation': 646, '#postcardsfromtheworld': 647, '#travelstroke': 648, '#JustAnotherDayInWA': 649, '#bmw': 650, '#gtr': 651, '#easyrecipes': 652, '#easyfood': 653, '#MarioTestino': 654, '#hijadecristo': 655, '#eatbetternotlesstravels': 656, '#igmasters': 657, '#estheticlabel': 658, '#veganfoodshare': 659, '#bookstagram': 660, '#eatmoreplants': 661, '#healthylife': 662, '#runnerspace': 663, '#guns': 664, '#pgdaily': 665, '#kirstentibballs': 666, '#callebaut': 667, '#StephenWilkes': 668, '#redcine': 669, '#ldfoundation': 670, '#Carinstagram': 671, '#kings_villages': 672, '#LGBTQI': 673, '#whereloveisillegal': 674, '#reebokcrossfit': 675, '#desafiofitem2017': 676, '#happyeaster': 677, '#live': 678, '#exittostreet': 679, '#glutenfree': 680, '#RECO_ig': 681, '#tb': 682, '#interiordecor': 683, '#livelife': 684, '#enjoy': 685, '#chamlive': 686, '#stayandwander': 687, '#travelawesome': 688, '#vscogram': 689, '#naturelovers': 690, '#iphone7': 691, '#springstyle': 692, '#magicalkenya': 693, '#nikonusa': 694, '#nikond5s': 695, '#oregon': 696, '#milano': 697, '#andreleonardo': 698, '#linkinbio': 699, '#japan': 700, '#USA': 701, '#SaltonSea': 702, '#carsbyaudisport': 703, '#ourplanetdaily': 704, '#thesummerexpedition': 705, '#PopUpPlayground': 706, '#Berlin': 707, '#travelgram': 708, '#ireland': 709, '#whpnaturalbeauty': 710, '#coffee': 711, '#vanlife': 712, '#g': 713, '#Easter': 714, '#abs': 715, '#cats': 716, '#fbf': 717, '#chicagofoodauthority': 718, '#toofacedxquay': 719, '#Leica': 720, '#lovefood': 721, '#thekitchn': 722, '#huffposttaste': 723, '#architect': 724, '#decor': 725, '#women': 726, '#moreocean': 727, '#fitnessfood': 728, '#birds': 729, '#EatFamous': 730, '#potd': 731, '#BMW': 732, '#fozazastyle': 733, '#garanceinjapan': 734, '#igersfrance': 735, '#bestdiscovery': 736, '#vienna': 737, '#earthofficial': 738, '#loves_paris': 739, '#parismaville': 740, '#hotelsandresorts': 741, '#Altamira': 742, '#ireneismaldives': 743, '#canon_photos': 744, '#bequadzilla': 745, '#mensphysique': 746, '#karinanigay': 747, '#finejewellery': 748, '#KERRentlyWearing': 749, '#sneakerfreakerfam': 750, '#sneakerlove': 751, '#bor': 752, '#TMM': 753, '#MommyAndMini2017': 754, '#fashionblogger': 755, '#super_france': 756, '#maserati': 757, '#badass': 758, '#menwithfootwear': 759, '#Mercurio': 760, '#Venus': 761, '#ltkunder50': 762, '#mkexplore': 763, '#splendid_shotz': 764, '#bestmeow': 765, '#veganrecipes': 766, '#govegan': 767, '#usatf': 768, '#IamATraveler': 769, '#savourschool': 770, '#EntrepreneurLife': 771, '#fashionworld': 772, '#fashiongirls': 773, '#nglive': 774, '#BCI': 775, '#bigcatsintiative': 776, '#weddingideas': 777, '#instabride': 778, '#weddinginspo': 779, '#guardiantravelsnaps': 780, '#projetoverao2017': 781, '#designporn': 782, '#czechoutmoravia': 783, '#wanderaround': 784, '#lovemylife': 785, '#indonesiaku': 786, '#resort': 787, '#nikonindonesia': 788, '#iamindonesia': 789, '#nikonasia': 790, '#mynikonlife': 791, '#nikonaustralia': 792, '#ITSistanbul': 793, '#vscodaily': 794, '#fw1718': 795, '#saveelephants': 796, '#retetielephants': 797, '#stoppoaching': 798, '#northernkenya': 799, '#whyilovekenya': 800, '#everydayafrica': 801, '#whale': 802, '#iceland': 803, '#girl': 804, '#danielwellington': 805, '#details': 806, '#pancakes': 807, '#winter': 808, '#peace': 809, '#aerial': 810, '#ltkunder100': 811, '#passion': 812, '#lifestyleblog': 813, '#thehappynow': 814, '#palmsprings': 815, '#zenfone3zoom': 816, '#captureomomento': 817, '#dubai': 818, '#tastingtable': 819, '#latergram': 820, '#movementtherapy': 821, '#designer': 822, '#womensbest': 823, '#exploreeverything': 824, '#liveinlevis': 825, '#cool': 826, '#light': 827, '#PanjiRasulullah': 828, '#foodporn': 829, '#movies': 830, '#parisjetaime': 831, '#topparisphoto': 832, '#onehairdressing': 833, '#fromwhereistand': 834, '#WhereLoveIsIllegal': 835, '#WalkingtheLAwash': 836, '#gotgvol2': 837, '#polishgirl': 838, '#nigay_ny': 839, '#hautejoaillerie': 840, '#thefateofthefurious': 841, '#sonyalphasclub': 842, '#theartofslowliving': 843, '#ford': 844, '#Huayra': 845, '#Luxury': 846, '#insadelicious': 847, '#manizha': 848, '#childofgod': 849, '#science': 850, '#inflowtravel': 851, '#streetmagazine': 852, '#hot_shotz': 853, '#moodytoning': 854, '#lifeandthyme': 855, '#Toronto': 856, '#balance': 857, '#flexible': 858, '#rescuefamily': 859, '#fashionpic': 860, '#fashiontips': 861, '#fashionphoto': 862, '#fashionhair': 863, '#fashionismo': 864, '#fashionbook': 865, '#fashionnews': 866, '#fashionbrand': 867, '#fashionshop': 868, '#fursforlife': 869, '#startwith1thing': 870, '#instawedding': 871, '#yourshotphotographer': 872, '#LoveFL': 873, '#thisisgoodcompany': 874, '#veganrecipe': 875, '#natgeohub': 876, '#TeamWycked': 877, '#dress': 878, '#happygirl': 879, '#party': 880, '#friends': 881, '#tv_buildings': 882, '#eattheworld': 883, '#travelwithme': 884, '#visitCZ': 885, '#ontheroad': 886, '#canon': 887, '#vietnam': 888, '#work': 889, '#sydney': 890, '#natureisspeaking': 891, '#india': 892, '#woman': 893, '#puertorico': 894, '#transformationtuesday': 895, '#alphacollective': 896, '#pretty': 897, '#outfit': 898, '#London': 899, '#america': 900, '#R8': 901, '#london': 902, '#BarcelonaCitizen': 903, '#igersbcn': 904, '#catalonia': 905, '#parkguell': 906, '#botswana': 907, '#nordstrom': 908, '#mountaingirls': 909, '#gopro': 910, '#carlifestyle': 911, '#blogdebelleza': 912, '#modernclassic': 913, '#bloglovinfood': 914, '#sonya7rii': 915, '#grateful': 916, '#E63S': 917, '#frankfurt': 918, '#chakra': 919, '#inabudhabi': 920, '#documentary': 921, '#newyorkcity': 922, '#cookbook': 923, '#jordan': 924, '#nakedplanet': 925, '#faafif': 926, '#MasirahPanjiRasulullah': 927, '#IslamRahmatanLilAlamin': 928, '#futuresolutionlx': 929, '#capetown': 930, '#LosAngeles': 931, '#topfrancephoto': 932, '#awesomepix': 933, '#natgeolandscape': 934, '#igersparis': 935, '#fashionstyling': 936, '#tat': 937, '#tatoo': 938, '#besttattoos': 939, '#ideastattoo': 940, '#blogger': 941, '#wbffpro': 942, '#highjewelry': 943, '#blackpepperenigma': 944, '#boostgod': 945, '#boostVIBES': 946, '#boostGANG': 947, '#modernnotoriety': 948, '#sonyimages': 949, '#PorscheGT3RS': 950, '#PaganiHuayra': 951, '#Zonda': 952, '#PaganiZonda': 953, '#makaniterror': 954, '#Jewelry': 955, '#vegetarian': 956, '#fashiondiaries': 957, '#Urano': 958, '#poetry': 959, '#modaparameninas': 960, '#instapic': 961, '#pracegover': 962, '#newdarlingsHOME': 963, '#weekly_feature': 964, '#fartoodope': 965, '#usaprimeshot': 966, '#master_shots': 967, '#royalsnappingartists': 968, '#transfer_visions': 969, '#philippemodel': 970, '#sunday': 971, '#whatveganseat': 972, '#ar15': 973, '#ourplanet': 974, '#Allshotsturkey': 975, '#bir_dakika': 976, '#gununkaresi': 977, '#hayatakarken': 978, '#turkinstagram': 979, '#zamanidurdur': 980, '#instasyon': 981, '#istanbuldayasam': 982, '#beniminsanlarim': 983, '#bugununkaresi': 984, '#ig_turkey': 985, '#turkishfollowers': 986, '#objektifimden': 987, '#my_dream_turkey': 988, '#gf_turkey': 989, '#igersturkey': 990, '#igersistanbul': 991, '#fashionnails': 992, '#fashionstudent': 993, '#breaky': 994, '#vscofashionfood': 995, '#edinburgh': 996, '#theworldguru': 997, '#MyFloridaisAdventure': 998, '#vegancommunity': 999}
    counts = Counter(corpus)
    counts = sorted(counts, key=counts.__getitem__,reverse=True)[:1000]
    best = sorted(zip(tag_to_num.keys(), _predictions), key=lambda x: x[1], reverse=True)[:5]
    keys, precentegaes = zip(*best)
    return " ".join(keys)

def predict_object(model, img, target_size, top_n=5):
    if img.size != target_size:
        img = img.resize(target_size)
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    _predictions = model.predict(x)
    return decode_predictions(_predictions, top=top_n)


if __name__=="__main__":
    arg_parser = argparse.ArgumentParser()
    arg_parser.add_argument("--image", help="path to image")
    args = arg_parser.parse_args()

    if args.image is None:
        arg_parser.print_help()
        sys.exit(1)

    if args.image is not None:
        img = Image.open(args.image)
        predictions = predict_object(model, img, target_size)
        print(predictions)

usage: ipykernel_launcher.py [-h] [--image IMAGE]
ipykernel_launcher.py: error: unrecognized arguments: -f /Users/orrbarkat/Library/Jupyter/runtime/kernel-162c4b9a-b1cf-423a-8b05-ac7173aead98.json


SystemExit: 2

/Users/orrbarkat/repos/hackaton_2017/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2870: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
